In [2]:
import pandas as pd
import os
import glob
import seaborn as sn
from sklearn.linear_model import LinearRegression

In [6]:
# setting up the path to read the datafiles
root = './data/*final.csv'
files = glob.glob(root)
print(files)

for file in files:
    df = pd.read_csv(file)
df

['./data/winequality-final.csv']


,type,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
0,red,11.6,0.580,0.66,2.2,0.074,10.0,47.0,1.00080,3.25,0.57,9.000000,3
1,red,8.3,1.020,0.02,3.4,0.084,6.0,11.0,0.99892,3.48,0.49,11.000000,3
2,red,10.4,0.610,0.49,2.1,0.200,5.0,16.0,0.99940,3.16,0.63,8.400000,3
3,red,7.3,0.980,0.05,2.1,0.061,20.0,49.0,0.99705,3.31,0.55,9.700000,3
4,red,6.8,0.815,0.00,1.2,0.267,16.0,29.0,0.99471,3.32,0.51,9.800000,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,white,6.6,0.340,0.40,8.1,0.046,68.0,170.0,0.99494,3.15,0.50,9.533333,6
6493,white,6.6,0.340,0.40,8.1,0.046,68.0,170.0,0.99494,3.15,0.50,9.533333,6
6494,white,6.0,0.410,0.23,1.1,0.066,22.0,148.0,0.99266,3.30,0.47,9.633333,5
6495,white,5.2,0.220,0.46,6.2,0.066,41.0,187.0,0.99362,3.19,0.42,9.733333,5
